In [1]:
import pandas as pd
import re
from typing import Tuple, Dict, FrozenSet
import logging
from datetime import datetime
import numpy as np

In [2]:
df = pd.read_csv('/root/sandbox/disa/data_merged_21-06-2023-10:36:19.tsv', sep='\t', header=0)


/tmp/ipykernel_3737247/1160699077.py:1: DtypeWarning: Columns (4,17,22,42,43,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,62,63,104,136,255,265,284,285,308,309,317,320,324,325,326,327,328,330,331,332,334,335,339,341,342,343,345,346,347,348,364,366,367,368,369,373,374,403,404,407,408,424,452,466,469,470,471,483,484,485,486,487,488,489,490,491,493,494,496,500,501,505,507,508,509,510,513,514,532,533,539,540,541,569,570,572,573,574,576,590,593,606,618,631,640,648,649,650,651,652,653,654,655,656,657,659,663,667,669,673,674,679,680,706,707,735,740,752,754,755,756,758,759,761,763,770,771,772,774,785,796,797,800,804,805,806,807,813,814,815,820,823,872,918,920,921,924,927,929,936,938,940,951,962,966,970,971,972,973,979,1213,1214,1216,1218,1222,1226,1227,1229,1236,1240,1261,1268,1270,1534,2350,2417,2471,2496,2508,2551,2578,2636) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/root/sandbox/disa/data_merged_21-06-2023-10:36:19.tsv', sep='\t', heade

In [3]:
df.columns

Index(['hhid', 'redcap_event_name', 's1_date', 's1_interviewer', 's1_csaid',
       's1_agriwork', 's1_gender', 's1_reside', 's1_dob', 's1_dobtype',
       ...
       'a5_transport___2_visit_2_arm_2', 'a5_land_nonagri___4_visit_2_arm_2',
       'a5_away_visit_2_arm_2', 'a5_nonfarm_who___999_visit_2_arm_2',
       'a5_loan_bank_decision_who___1_visit_2_arm_2',
       'a5_shg_participate_visit_2_arm_2',
       'a5_loan_shg_decision_who___4_visit_2_arm_2', 'a10_1030_visit_2_arm_2',
       'a10_2100_visit_2_arm_2', 'a10_1000_visit_2_arm_2'],
      dtype='object', length=2843)

In [4]:
df['s4_hhcategory'].unique()

array(['NPM/Transition', 'Conventional',
       'S2S (organic in at least one plot)', nan,
       'Used to be organic, but no longer'], dtype=object)

In [5]:
condition = (df['s4_hhcategory'] == '') | (df['s4_hhcategory'] == 'Used to be organic, but no longer') 
len(df[condition])

7

In [6]:
print(len(df[df['s4_hhcategory'].isna()]))
print(len(df[df['s4_hhcategory'].isnull()]))
print(len(df[pd.isna(df['s4_hhcategory'])]))

24
24
24


In [7]:
change_count = len(df[df['s4_hhcategory'] == "Used to be organic, but no longer"])
null_count = pd.isna(df['s4_hhcategory']).sum()
blank_count = len(df[df['s4_hhcategory']== ""] )
print(change_count, null_count, blank_count)

7 24 0


In [8]:
pd.isna(df['s4_hhcategory'])

0       False
1       False
2       False
3       False
4       False
        ...  
1245    False
1246    False
1247    False
1248    False
1249    False
Name: s4_hhcategory, Length: 1250, dtype: bool

In [9]:
filtered_df = df.filter(like="_gp_", axis=1)
filtered_df

,s4_gp_ka1,s4_gp_ka2,s4_gp_ka3,s4_gp_an1,s4_gp_ku1
0,NaN,NaN,NaN,Kurli,NaN
1,NaN,NaN,NaN,Kurli,NaN
2,NaN,NaN,NaN,Kurli,NaN
3,NaN,NaN,NaN,Kurli,NaN
4,NaN,NaN,NaN,Kurli,NaN
...,...,...,...,...,...
1245,NaN,T Velamvaripalli,NaN,NaN,NaN
1246,NaN,T Velamvaripalli,NaN,NaN,NaN
1247,NaN,T Velamvaripalli,NaN,NaN,NaN
1248,NaN,T Velamvaripalli,NaN,NaN,NaN


In [10]:
# df['s4_cluster'] = df['s4_cluster_ka'].fillna(df['s4_cluster_an'].fillna(df['s4_cluster_ku']))
# final = df.filter(like="cluster",axis=1)
# final

In [11]:
df['h3_crop1otherpaddy'].unique()

array([nan, 'JJL'], dtype=object)

In [12]:
print(df["h1_exp_water_amt"])
df["h1_exp_water_amt"] = df["h1_exp_water_amt"].div(6, fill_value=np.nan)
print(df["h1_exp_water_amt"])

0       1200.0
1       1200.0
2       1200.0
3       1800.0
4       1200.0
         ...  
1245       NaN
1246       NaN
1247       NaN
1248       NaN
1249       NaN
Name: h1_exp_water_amt, Length: 1250, dtype: float64
0       200.0
1       200.0
2       200.0
3       300.0
4       200.0
        ...  
1245      NaN
1246      NaN
1247      NaN
1248      NaN
1249      NaN
Name: h1_exp_water_amt, Length: 1250, dtype: float64


In [13]:
df["h1_income_total"] = df[
        [
            "h1_income_cultivation",
            "h1_income_cultivation_amt",
            "h1_income_livestock",
            "h1_income_livestock_amt",
            "h1_income_othagri",
            "h1_income_othagri_amt",
            "h1_income_business",
            "h1_income_nonagri_amt",
            "h1_income_wages",
            "h1_income_wages_amt",
            "h1_income_salaries",
            "h1_income_salaries_amt",
            "h1_income_pension",
            "h1_income_pension_amt",
            "h1_income_govt",
            "h1_income_govt_amt",
            "h1_income_remittances",
            "h1_income_remittances_amt",
            "h1_income_other",
            "h1_income_other_specify",
            "h1_income_other_amt",
        ]
    ].sum(axis=1, skipna=True)

/tmp/ipykernel_3737247/379117470.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["h1_income_total"] = df[


In [14]:
income_df = df[
        [
            "h1_income_cultivation",
            "h1_income_cultivation_amt",
            "h1_income_livestock",
            "h1_income_livestock_amt",
            "h1_income_othagri",
            "h1_income_othagri_amt",
            "h1_income_business",
            "h1_income_nonagri_amt",
            "h1_income_wages",
            "h1_income_wages_amt",
            "h1_income_salaries",
            "h1_income_salaries_amt",
            "h1_income_pension",
            "h1_income_pension_amt",
            "h1_income_govt",
            "h1_income_govt_amt",
            "h1_income_remittances",
            "h1_income_remittances_amt",
            "h1_income_other",
            "h1_income_other_specify",
            "h1_income_other_amt",
        ]
    ]

income_df

,h1_income_cultivation,h1_income_cultivation_amt,h1_income_livestock,h1_income_livestock_amt,h1_income_othagri,h1_income_othagri_amt,h1_income_business,h1_income_nonagri_amt,h1_income_wages,h1_income_wages_amt,...,h1_income_salaries_amt,h1_income_pension,h1_income_pension_amt,h1_income_govt,h1_income_govt_amt,h1_income_remittances,h1_income_remittances_amt,h1_income_other,h1_income_other_specify,h1_income_other_amt
0,No,NaN,No,NaN,No,NaN,Yes,70000.0,No,NaN,...,NaN,Yes,15000.0,Yes,7500.0,No,NaN,No,NaN,NaN
1,No,NaN,No,NaN,No,NaN,Yes,25000.0,No,NaN,...,NaN,Yes,15000.0,Yes,7500.0,No,NaN,No,NaN,NaN
2,No,NaN,No,NaN,No,NaN,No,NaN,No,NaN,...,NaN,No,NaN,Yes,7500.0,No,NaN,No,NaN,NaN
3,No,NaN,Yes,50000.0,No,NaN,No,NaN,No,NaN,...,NaN,No,NaN,Yes,7500.0,No,NaN,No,NaN,NaN
4,No,NaN,No,NaN,No,NaN,No,NaN,Yes,65000.0,...,42000.0,Yes,15000.0,Yes,7500.0,No,NaN,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,No,NaN,No,NaN,No,NaN,No,NaN,Yes,55000.0,...,NaN,Yes,2500.0,Yes,5000.0,No,NaN,No,NaN,NaN
1246,No,NaN,No,NaN,No,NaN,No,NaN,No,NaN,...,NaN,Yes,2500.0,Yes,5000.0,No,NaN,No,NaN,NaN
1247,No,NaN,No,NaN,No,NaN,No,NaN,Yes,55000.0,...,NaN,Yes,2500.0,Yes,5000.0,No,NaN,No,NaN,NaN
1248,No,NaN,No,NaN,No,NaN,No,NaN,Yes,55000.0,...,NaN,No,NaN,No,NaN,No,NaN,No,NaN,NaN


In [15]:
def convert_text_to_months(text):
    pattern = r'\d+\s*[m,M]onth'
    match = re.match(pattern, text)
    if match is None:
        raise Exception("Could not extract month length from text: ", text)
    return int(match.group(0).split()[0])

In [16]:
# convert_text_to_months("Approx. 1 month")

In [17]:
str_pattern = r'.*\d+'
pattern = re.compile(str_pattern)
res = pattern.search("Approx. 3 Month")
print(res.group(0))



Approx. 3


In [18]:
df["h1_exp_phone_unit"].unique()

array(['Approx. 1 month', nan, 'Approx. 2 months', 'Approx. 3 months',
       'Other', 'Approx. 1 year'], dtype=object)

In [19]:
out = df[df["h1_exp_phone_unit"] == 'Other']
out

,hhid,redcap_event_name,s1_date,s1_interviewer,s1_csaid,s1_agriwork,s1_gender,s1_reside,s1_dob,s1_dobtype,...,a5_land_nonagri___4_visit_2_arm_2,a5_away_visit_2_arm_2,a5_nonfarm_who___999_visit_2_arm_2,a5_loan_bank_decision_who___1_visit_2_arm_2,a5_shg_participate_visit_2_arm_2,a5_loan_shg_decision_who___4_visit_2_arm_2,a10_1030_visit_2_arm_2,a10_2100_visit_2_arm_2,a10_1000_visit_2_arm_2,h1_income_total
710,KA-TP-SP-RP-3033,visit_1_arm_1,9-Jul-22,Abhishekam,999,Yes,Male,Yes,1-Jan-48,Based on Aadhar card or other government-issue...,...,Unchecked,When,Unchecked,Checked,yes,Unchecked,NaN,NaN,NaN,78000.0
1213,ka-tv-tv-gp-1066,visit_1_arm_1,12-Jul-22,Jyothi,999,Yes,Male,Yes,7-Jan-78,Based on Aadhar card or other government-issue...,...,Unchecked,When,Unchecked,Unchecked,yes,Unchecked,NaN,NaN,NaN,38000.0


In [20]:
df[df["h1_exp_phone_unit"] == "Other"]['hhid']

710     KA-TP-SP-RP-3033
1213    ka-tv-tv-gp-1066
Name: hhid, dtype: object

In [21]:
pattern = r'a10_\d{4}'

filtered_columns = [column for column in list(df.columns) if re.match(pattern, column)]
df_slice = df[filtered_columns].copy()
df_slice.fillna("Other", inplace=True)

df[filtered_columns] = df_slice
# # df[filtered_columns].fillna("Other", inplace=True)
# # timeuse = set()
# # for column in filtered_columns.columns:
# #     timeuse |= set(list(filtered_columns[column].unique()))

# # timeuse
# newdf
# print(newdf)

nan_counts = df[filtered_columns].isna().sum()
print(nan_counts)

a10_1630_visit_1_arm_1    0
a10_2330_visit_1_arm_1    0
a10_2400_visit_1_arm_1    0
a10_1930_visit_1_arm_1    0
a10_0630_visit_1_arm_1    0
                         ..
a10_0230_visit_2_arm_2    0
a10_1430_visit_2_arm_2    0
a10_1030_visit_2_arm_2    0
a10_2100_visit_2_arm_2    0
a10_1000_visit_2_arm_2    0
Length: 240, dtype: int64


In [22]:
df.columns

Index(['hhid', 'redcap_event_name', 's1_date', 's1_interviewer', 's1_csaid',
       's1_agriwork', 's1_gender', 's1_reside', 's1_dob', 's1_dobtype',
       ...
       'a5_land_nonagri___4_visit_2_arm_2', 'a5_away_visit_2_arm_2',
       'a5_nonfarm_who___999_visit_2_arm_2',
       'a5_loan_bank_decision_who___1_visit_2_arm_2',
       'a5_shg_participate_visit_2_arm_2',
       'a5_loan_shg_decision_who___4_visit_2_arm_2', 'a10_1030_visit_2_arm_2',
       'a10_2100_visit_2_arm_2', 'a10_1000_visit_2_arm_2', 'h1_income_total'],
      dtype='object', length=2844)

In [23]:
pattern = r"a10_\d{4}(_vist_(\d+)_arm_(\d+))?"


filtered_columns = [column for column in df.columns if re.match(pattern, column)]

event_bin = {}


for column in filtered_columns:
    match = re.match(pattern, column)
    event_name = match.group(1)
    if event_name in event_bin:
        event_bin[event_name].append(column)
    else:
        event_bin[event_name] = [column]
        

f = open("event_counts_work.txt", "w")
for event_name, column_names in event_bin.items():
    nan_counts = (df.sort_index(axis=1))[column_names].isna().sum()
    f.write(f"For Event Name: {event_name}\n")
    f.write(f"{str(nan_counts)}\n")

f.close()



In [24]:
event_bin

{None: ['a10_1630_visit_1_arm_1',
  'a10_2330_visit_1_arm_1',
  'a10_2400_visit_1_arm_1',
  'a10_1930_visit_1_arm_1',
  'a10_0630_visit_1_arm_1',
  'a10_1300_visit_1_arm_1',
  'a10_0800_visit_1_arm_1',
  'a10_0930_visit_1_arm_1',
  'a10_1130_visit_1_arm_1',
  'a10_1530_visit_1_arm_1',
  'a10_2130_visit_1_arm_1',
  'a10_2300_visit_1_arm_1',
  'a10_0030_visit_1_arm_1',
  'a10_1400_visit_1_arm_1',
  'a10_1730_visit_1_arm_1',
  'a10_0830_visit_1_arm_1',
  'a10_2000_visit_1_arm_1',
  'a10_0530_visit_1_arm_1',
  'a10_0200_visit_1_arm_1',
  'a10_0400_visit_1_arm_1',
  'a10_0730_visit_1_arm_1',
  'a10_0500_visit_1_arm_1',
  'a10_1830_visit_1_arm_1',
  'a10_0900_visit_1_arm_1',
  'a10_0100_visit_1_arm_1',
  'a10_0330_visit_1_arm_1',
  'a10_0130_visit_1_arm_1',
  'a10_0300_visit_1_arm_1',
  'a10_0700_visit_1_arm_1',
  'a10_1800_visit_1_arm_1',
  'a10_1200_visit_1_arm_1',
  'a10_2230_visit_1_arm_1',
  'a10_0430_visit_1_arm_1',
  'a10_1330_visit_1_arm_1',
  'a10_1700_visit_1_arm_1',
  'a10_0600_vi

In [25]:
for column in filtered_columns:
    print(column)

a10_1630_visit_1_arm_1
a10_2330_visit_1_arm_1
a10_2400_visit_1_arm_1
a10_1930_visit_1_arm_1
a10_0630_visit_1_arm_1
a10_1300_visit_1_arm_1
a10_0800_visit_1_arm_1
a10_0930_visit_1_arm_1
a10_1130_visit_1_arm_1
a10_1530_visit_1_arm_1
a10_2130_visit_1_arm_1
a10_2300_visit_1_arm_1
a10_0030_visit_1_arm_1
a10_1400_visit_1_arm_1
a10_1730_visit_1_arm_1
a10_0830_visit_1_arm_1
a10_2000_visit_1_arm_1
a10_0530_visit_1_arm_1
a10_0200_visit_1_arm_1
a10_0400_visit_1_arm_1
a10_0730_visit_1_arm_1
a10_0500_visit_1_arm_1
a10_1830_visit_1_arm_1
a10_0900_visit_1_arm_1
a10_0100_visit_1_arm_1
a10_0330_visit_1_arm_1
a10_0130_visit_1_arm_1
a10_0300_visit_1_arm_1
a10_0700_visit_1_arm_1
a10_1800_visit_1_arm_1
a10_1200_visit_1_arm_1
a10_2230_visit_1_arm_1
a10_0430_visit_1_arm_1
a10_1330_visit_1_arm_1
a10_1700_visit_1_arm_1
a10_0600_visit_1_arm_1
a10_2200_visit_1_arm_1
a10_1230_visit_1_arm_1
a10_1100_visit_1_arm_1
a10_2030_visit_1_arm_1
a10_1600_visit_1_arm_1
a10_1900_visit_1_arm_1
a10_1500_visit_1_arm_1
a10_0230_vi

In [26]:
from pipeline.cleanup_a10 import calculate_work_totals


df_new = calculate_work_totals(df)

df_new.filter(regex=r'total_.*')

summation headers:
['total_non_work_visit_2_arm_1', 'total_non_work_visit_3_arm_1', 'total_non_work_visit_2_arm_2', 'total_non_work_visit_1_arm_1', 'total_non_work_visit_1_arm_2', 'total_agri_work_visit_2_arm_1', 'total_agri_work_visit_3_arm_1', 'total_agri_work_visit_2_arm_2', 'total_agri_work_visit_1_arm_1', 'total_agri_work_visit_1_arm_2', 'total_other_work_visit_2_arm_1', 'total_other_work_visit_3_arm_1', 'total_other_work_visit_2_arm_2', 'total_other_work_visit_1_arm_1', 'total_other_work_visit_1_arm_2']


,total_non_work_visit_2_arm_1,total_non_work_visit_3_arm_1,total_non_work_visit_2_arm_2,total_non_work_visit_1_arm_1,total_non_work_visit_1_arm_2,total_agri_work_visit_2_arm_1,total_agri_work_visit_3_arm_1,total_agri_work_visit_2_arm_2,total_agri_work_visit_1_arm_1,total_agri_work_visit_1_arm_2,total_other_work_visit_2_arm_1,total_other_work_visit_3_arm_1,total_other_work_visit_2_arm_2,total_other_work_visit_1_arm_1,total_other_work_visit_1_arm_2
0,24.0,24.0,24.0,13.5,11.5,0,0,0,0.0,0.0,0,0,0,10.5,12.5
1,24.0,24.0,24.0,13.5,14.0,0,0,0,0.0,0.0,0,0,0,10.5,10.0
2,24.0,24.0,24.0,12.5,11.5,0,0,0,10.0,7.5,0,0,0,1.5,5.0
3,24.0,24.0,24.0,11.5,11.5,0,0,0,11.5,8.0,0,0,0,1.0,4.5
4,24.0,24.0,24.0,15.0,13.5,0,0,0,1.5,0.0,0,0,0,7.5,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,24.0,24.0,24.0,13.5,12.0,0,0,0,10.5,9.5,0,0,0,0.0,2.5
1246,24.0,24.0,24.0,13.5,12.5,0,0,0,10.5,9.0,0,0,0,0.0,2.5
1247,24.0,24.0,24.0,12.5,13.5,0,0,0,8.5,10.5,0,0,0,3.0,0.0
1248,24.0,24.0,24.0,14.0,12.5,0,0,0,10.0,9.5,0,0,0,0.0,2.0


In [27]:
print(filtered_columns)

['a10_1630_visit_1_arm_1', 'a10_2330_visit_1_arm_1', 'a10_2400_visit_1_arm_1', 'a10_1930_visit_1_arm_1', 'a10_0630_visit_1_arm_1', 'a10_1300_visit_1_arm_1', 'a10_0800_visit_1_arm_1', 'a10_0930_visit_1_arm_1', 'a10_1130_visit_1_arm_1', 'a10_1530_visit_1_arm_1', 'a10_2130_visit_1_arm_1', 'a10_2300_visit_1_arm_1', 'a10_0030_visit_1_arm_1', 'a10_1400_visit_1_arm_1', 'a10_1730_visit_1_arm_1', 'a10_0830_visit_1_arm_1', 'a10_2000_visit_1_arm_1', 'a10_0530_visit_1_arm_1', 'a10_0200_visit_1_arm_1', 'a10_0400_visit_1_arm_1', 'a10_0730_visit_1_arm_1', 'a10_0500_visit_1_arm_1', 'a10_1830_visit_1_arm_1', 'a10_0900_visit_1_arm_1', 'a10_0100_visit_1_arm_1', 'a10_0330_visit_1_arm_1', 'a10_0130_visit_1_arm_1', 'a10_0300_visit_1_arm_1', 'a10_0700_visit_1_arm_1', 'a10_1800_visit_1_arm_1', 'a10_1200_visit_1_arm_1', 'a10_2230_visit_1_arm_1', 'a10_0430_visit_1_arm_1', 'a10_1330_visit_1_arm_1', 'a10_1700_visit_1_arm_1', 'a10_0600_visit_1_arm_1', 'a10_2200_visit_1_arm_1', 'a10_1230_visit_1_arm_1', 'a10_1100_v

In [28]:
pattern = r"a10_\d{4}(_visit_(\d+)_arm_(\d+))?"
match = re.match(pattern, 'a10_1630_visit_1_arm_1')
print(match.groups())

('_visit_1_arm_1', '1', '1')


In [29]:
df['hhid']

0       AT-GT-KR-KP-2168
1       AT-GT-KR-KP-2169
2       AT-GT-KR-KP-2170
3       AT-GT-KR-KP-2171
4       AT-GT-KR-KP-2172
              ...       
1245    ka-Tv-Tv-Nt-1453
1246    ka-Tv-Tv-Nt-1454
1247    ka-Tv-Tv-Nt-1455
1248    ka-Tv-Tv-Nt-1456
1249    ka-Tv-Tv-Nt-1457
Name: hhid, Length: 1250, dtype: object

In [30]:
df["h1_income_other_specify"].unique()

array([nan, '24000', 'NREGS', 'Driver', 'Tractor', 'Tailor', 'Auto',
       'Lari Driver', 'Lari driver', 'Millets&bullero', 'Bullero (auto)',
       'Lari', 'Dairy', 'Tracter', 'RMP Docter',
       'Chakali ( battalu wash)', 'Contract worker', 'Ammavadi ',
       'Tailoring', 'Track tar', 'Auto '], dtype=object)

In [32]:
from pipeline.cleanup_h1 import reassign_h1_income_other_specify


df = reassign_h1_income_other_specify(df)

KeyError: 'h1_income_other_specify'